In [ ]:
# fix seeds 
SEED = 42

import random as rn
rn.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

import sys
sys.path.append("..")

In [ ]:
from src.dataset.segmentation_dataset import SegmentationDataset

s = SegmentationDataset(
    '/workspace/data/prepped/segmentation-train-10.tfrecord',
    one_image_label=True,
    #loss_function='xent',
    nb_class=5,
    batch_size=128,
    shuffle_buffer_size=0
)
dataset = s.dataset
w = s.class_weights

for b in dataset:
    for p in b:
        print(p.shape)


In [ ]:
from src.model.unet import Unet
import segmentation_models as sm
import tensorflow as tf

model = Unet(
    input_shape=(128, 128, 10),
    classes=5,  
    encoder_weights='test-weights.h5',
    encoder_freeze=True,
    one_image_label=True
)
model.compile(
    optimizer='adam',
    loss=sm.losses.categorical_focal_loss,
    #loss=sm.losses.CategoricalCELoss(class_weights=w), 
    #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # only for one image label
    metrics=[
        'categorical_accuracy', 
        #sm.metrics.iou_score,
    ], 
)
model.fit(
    dataset, 
    epochs=1,
    #class_weight=w, ## only for one_image_label
#     validation_data=val_dataset,
#     validation_steps=2,
#     callbacks=[
#         tf.keras.callbacks.CSVLogger('logs/metrics.csv'), 
#         tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5), 
#         tf.keras.callbacks.ReduceLROnPlateau(), 
#         tf.keras.callbacks.TensorBoard('logs/test', profile_batch=0)  
#     ],
    workers=4,
    use_multiprocessing=True
)

In [ ]:
# results = model.evaluate(train_dataset)
# print("Loss :", results[0])
# print("Accuracy :", results[1])
preds = model.predict(dataset)
print(preds.shape)

